# Homework #10:
|Author| Stanley A. Baronett|
|--|-------------------------------|
|Created | 11/2/2019|
|Updated | 11/2/2019|

## 15.5 Local Optimization I

We wish to find the ground state energy between three neutral atoms or molecules, $N=3$. To simplify the calculation, we'll use the _Lennard-Jones (LJ) potential_:

$$ V = 4\varepsilon \big[ \big(\frac{\sigma}{r}\big)^{12} - \big(\frac{\sigma}{r}\big)^{6} \big] \tag{1},$$

where $\varepsilon$ is the depth of the potential well, $\sigma$ is the finite distance at which the interparticle potential is zero, and $r$ is the distance between the particles. We will also assume $\varepsilon$ = $\sigma$ = 1.

In [1]:
def LJ(r):
    """
    Calculate and return the Lennard-Jones potential between two
    atoms as a function of separation distance.
    Input:
        r: interatomic separation distance.
    Output:
        Lennard-Jones potential with ε = σ = 1
    """
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

Since $N=3$, the number of interatomic pairs will be

$$ \frac{N\times(N-1)}{2} = 3 \tag{2}.$$

Thus, we'll need to calculate the distance $r$ for each pair and evaluate its corresponding LJ$_3$ potential.

In [2]:
import numpy as np

def total_LJ_V(positions):
    """
    Calculate the total potential energy of N number of atoms using
    the Lennard-Jones potential.
    Input:
        positions: 3*N 1-D array which represents the atomic positions
                   (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    Output:
        E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    for i in range(N_atom - 1):
        for j in range(i + 1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            dist = np.linalg.norm(pos1 - pos2)
            #print(i,j, dist)
            E += LJ(dist)
    return E

Now that we have a way to calculate the total energy of $N$ atoms, using the LJ potential, we now need to randomly generate positions for these $N$ atoms. To do so, we'll generate a random sample for the $x, y,$ and $z$ coordinates of each atom to be within some cubic box with set length $L$.

In [3]:
def init_pos(N, L=2):
    """
    Randomly initialize the 3-D position of N atoms within a cube
    of length L.
    Input:
        N: the number of atoms
        L: 3-D boundary of random sample (e.g., length of cube)
    Output:
        3*N 1-D array of atomic positions
        (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    """
    return L*np.random.random_sample((N*3,))

In [4]:
pos = np.array([ 0.7430002202,        0.2647603899,       -0.0468575389,
                -0.7430002647,       -0.2647604843,        0.0468569750,
                 0.1977276118,       -0.4447220146,        0.6224700350,
                -0.1977281310,        0.4447221826,       -0.6224697723,
                -0.1822009635,        0.5970484122,        0.4844363476,
                 0.1822015272,       -0.5970484858,       -0.4844360463])
total_LJ_V(pos)

-12.712062256782637

In [11]:
# Visulization of the LJ clusters
from ase.visualize import view
from ase import Atoms
N = 6
cluster = Atoms('N'+str(N), positions=np.reshape(pos*2.0,[N,3]))
view(cluster, viewer='x3d')

In [14]:
view(cluster, viewer='ngl')

In [15]:
v=_
v.custom_colors({'Mn':'green','As':'blue'})
v.view._remote_call("setSize", target="Widget", args=["400px", "400px"])
v.view.center_view()
v.view.background='#ffc'
v.view.parameters=dict(clipDist=-200)